# 6.3 Geographical Visualzation in Python

## 1. Imports, Settings, and Loading the Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pyarrow
import warnings
import folium
import json

In [2]:
project_path = r'C:\Users\cneva\Projects\NBA Shot Analysis'
df = pd.read_csv(os.path.join(project_path,'Datasets','NBA_Shot_Analysis.csv'))
pd.set_option("display.max_columns", None) # pd.reset_option("display.max_columns")
warnings.filterwarnings('ignore', category = FutureWarning) # Suppress seaborn specific future warnings

In [3]:
df.sample(10)

,season,game_id,game_date,arena,city,state,country,player_team_name,player_team_abb,is_home_game,opponent_team_name,opponent_team_abb,player_name,position_group,position,salary,shot_made,action_category,action_type,shot_type,basic_zone,zone_name,zone_range,loc_x,loc_y,shot_distance,quarter,mins_left,secs_left,secs_left_in_quarter,secs_elapsed_in_game
168664,2024,22300644,2024-01-27,Ball Arena,Denver,Colorado,USA,Denver Nuggets,DEN,True,Philadelphia 76ers,PHI,Nikola Jokic,C,C,47607350,True,layup,Cutting Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,0.1,7.05,1,2,0,31,31,751
158121,2024,22300020,2023-11-14,Barclays Center,Brooklyn,New York,USA,Brooklyn Nets,BKN,True,Orlando Magic,ORL,Mikal Bridges,F,SF,21700000,True,shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side,24+ ft.,23.5,2.45,23,3,7,36,456,396
212443,2024,22300728,2024-02-07,Capital One Arena,"Washington, D.C.",NaN,USA,Washington Wizards,WAS,True,Cleveland Cavaliers,CLE,Tyus Jones,G,PG,14000000,True,shot,Pullup Jump shot,3PT Field Goal,Above the Break 3,Right Side Center,24+ ft.,-16.7,26.85,27,4,0,31,31,871
112355,2024,22301129,2024-04-06,FedExForum,Memphis,Tennessee,USA,Philadelphia 76ers,PHI,False,Memphis Grizzlies,MEM,Joel Embiid,C,C,47607350,True,shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side,8-16 ft.,8.3,7.55,8,2,11,29,689,89
57791,2024,22301207,2023-12-06,Scotiabank Arena,Toronto,Ontario,Canada,Toronto Raptors,TOR,True,Miami Heat,MIA,Dennis Schroder,G,PG,12405000,False,shot,Jump Shot,3PT Field Goal,Above the Break 3,Center,24+ ft.,2.3,30.35,25,4,3,59,239,719
5489,2024,22300692,2024-02-02,Toyota Center,Houston,Texas,USA,Houston Rockets,HOU,True,Toronto Raptors,TOR,Amen Thompson,F,SF,8809284,True,layup,Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center,Less Than 8 ft.,3.3,7.95,4,2,5,46,346,466
58863,2024,22300124,2023-11-01,American Airlines Center,Dallas,Texas,USA,Dallas Mavericks,DAL,True,Chicago Bulls,CHI,Derrick Jones Jr.,F,SF,2709849,False,shot,Jump Shot,3PT Field Goal,Right Corner 3,Right Side,24+ ft.,-22.7,10.85,23,1,11,40,700,40
203141,2024,22300382,2023-12-22,Wells Fargo Center,Philadelphia,Pennsylvania,USA,Philadelphia 76ers,PHI,True,Toronto Raptors,TOR,Tobias Harris,F,PF,39270150,False,layup,Driving Layup Shot,2PT Field Goal,Restricted Area,Center,Less Than 8 ft.,-0.0,7.35,2,4,9,37,577,337
154087,2024,22300164,2023-11-08,FedExForum,Memphis,Tennessee,USA,Memphis Grizzlies,MEM,True,Miami Heat,MIA,Marcus Smart,G,PG,18583713,False,shot,Pullup Jump shot,2PT Field Goal,In The Paint (Non-RA),Center,Less Than 8 ft.,-2.8,8.85,4,2,4,28,268,508
148836,2024,22300952,2024-03-13,American Airlines Center,Dallas,Texas,USA,Dallas Mavericks,DAL,True,Golden State Warriors,GSW,Luka Doncic,G,PG,40064220,False,shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center,24+ ft.,-13.4,27.85,26,2,7,40,460,340


## Preparing the Data for Visualization 

In [4]:
df_team = (df \
           .loc[:,['player_team_name', 'arena', 'city', 'state', 'country', 'shot_made']]
           .groupby(['player_team_name', 'arena', 'city', 'state', 'country'])
           .agg(field_goal_perc = ('shot_made', 'mean'))
           .reset_index()
          )

df_team.sample(10)

,player_team_name,arena,city,state,country,field_goal_perc
614,Orlando Magic,FedExForum,Memphis,Tennessee,USA,0.469136
479,Milwaukee Bucks,Little Caesars Arena,Detroit,Michigan,USA,0.497207
778,San Antonio Spurs,Target Center,Minneapolis,Minnesota,USA,0.420765
523,New Orleans Pelicans,Ball Arena,Denver,Colorado,USA,0.486034
636,Orlando Magic,Wells Fargo Center,Philadelphia,Pennsylvania,USA,0.452632
556,New York Knicks,Delta Center,Salt Lake City,Utah,USA,0.436170
103,Charlotte Hornets,Madison Square Garden,New York,New York,USA,0.442623
728,Sacramento Kings,Crypto.com Arena,Los Angeles,California,USA,0.528497
305,Houston Rockets,Little Caesars Arena,Detroit,Michigan,USA,0.455556
318,Houston Rockets,Wells Fargo Center,Philadelphia,Pennsylvania,USA,0.500000


In [5]:
df_team_example = (df_team \
                   .query("player_team_name == 'New York Knicks'")
                   .loc[:,['state', 'field_goal_perc']]
                  )
df_team_example.sample(10)

,state,field_goal_perc
571,Louisiana,0.366667
563,California,0.468085
553,New York,0.464088
572,North Carolina,0.514286
574,Massachusetts,0.473684
570,Ontario,0.543478
562,California,0.430233
564,Florida,0.460526
576,Texas,0.456522
558,Wisconsin,0.458182


## 3. Graphical Visualization 

In [6]:
geo_json = project_path + r'\us-states.json'

In [7]:
map = folium.Map(location = [40, -95], zoom_start = 4.5)

folium.Choropleth(
    geo_data = geo_json, 
    data = df_team_example,
    columns = ['state', 'field_goal_perc'],
    key_on = 'feature.properties.name',
    fill_color = 'Oranges', 
    fill_opacity = 1.0, 
    line_opacity=0.5,
    nan_fill_color = 'white',
    legend_name = "field_goal_perc"
).add_to(map)

folium.LayerControl().add_to(map)

map